In [1]:
import warnings
warnings.filterwarnings("ignore")
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tggan import *
from utils import *
import tensorflow as tf


2020-06-04 05:54:39,335 - INFO - is GPU available? True
/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train_edges = np.loadtxt('outputs-scale-free-nodes-100-samples-200/scale-free-nodes-100-samples-200_train.txt')
train_edges = np.loadtxt('outputs-scale-free-nodes-100-samples-200/scale-free-nodes-100-samples-200_train.txt')

In [3]:
n_nodes = 100
rw_len = 8
t_end = 1.
t0 = 0.1
edge_contact_time = t0*0.5
batch_size = 32
gpu_id = 0
embedding_size = 32
lr = 0.003
use_decoder = 'deep'
constraint_method = 'min_max'
scale = 0.1
init_walk_method = 'uniform'
use_wgan = True
n_eval_loop = 25

In [4]:
walker = TemporalWalker(n_nodes, train_edges, t_end, scale, rw_len, batch_size, init_walk_method=init_walk_method)
print(walker.walk().__next__().shape)

(32, 9, 3)


In [5]:
tggan = TGGAN(N=n_nodes, rw_len=rw_len,
                      t_end=t_end,
                      edge_contact_time=edge_contact_time,
                      walk_generator=walker.walk, batch_size=batch_size, gpu_id=gpu_id,
                      noise_type="Gaussian",
                      disc_iters=3,
                      W_down_discriminator_size=embedding_size,
                      W_down_generator_size=embedding_size,
                      l2_penalty_generator=1e-7,
                      l2_penalty_discriminator=5e-5,
                      generator_x_up_layers=[64],
                      generator_t0_up_layers=[128],
                      generator_tau_up_layers=[128],
                      generator_layers=[100, 20],
                      discriminator_layers=[80, 20],
                      temp_start=5,
                      learning_rate=lr,
                      use_gumbel=True,
                      use_wgan=use_wgan,
                      wasserstein_penalty=10,
                      use_decoder=use_decoder,
                      constraint_method=constraint_method,
                      )

In [6]:
saver = tf.train.Saver()
saver.restore(tggan.session, 
                  "snapshots-scale-free-nodes-100-samples-200/simulation-_iter_10.ckpt")

In [8]:
eval_transitions=1e6
p = 10
n_smpls = tggan.batch_size * p
# n_eval_iters = int(eval_transitions / n_smpls)
n_eval_iters = 1

sample_many = tggan.generate_discrete(batch_size*10, reuse=True, 
                                      edge_contact_time=edge_contact_time,n_eval_loop=n_eval_loop)
fake_graphs, fake_x_t0 = tggan.get_fake_graphs(sample_many, n_smpls, n_eval_iters, n_eval_loop)

2020-06-04 05:56:29,188 - INFO - eval_iters: 0 eval_loop: 22
2020-06-04 05:56:29,190 - INFO - eval node logit min: -1.1656744480133057 max: 1.4816631078720093
2020-06-04 05:56:29,191 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [62 79 87 82 32 95 55 83 58 16 15 76 81 91 92  5], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:56:29,192 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [49 40  3 12 26 39  0 87 75 50 98 28 97 26 91 14], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:56:29,194 - INFO - eval_iters: 0 eval_loop: 23
2020-06-04 05:56:29,195 - INFO - eval node logit min: -1.1656744480133057 max: 1.4816631078720093
2020-06-04 05:56:29,196 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [87 82 32 95 55 83 58 16 15 76 81 91 92  5  0 24], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:56:29,197 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [ 3 12 26 39  0 87 75 50 98 28 97 26 91 14 74 76], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:56:29,200 - INFO - eval_iters: 0 eval_loop: 24
2020-

In [13]:
print(convert_graphs(fake_graphs).shape)

(2530, 4)


In [11]:
eval_transitions=1e6
p = 10
n_smpls = tggan.batch_size * p
# n_eval_iters = int(eval_transitions / n_smpls)
n_eval_iters = 1

sample_many = tggan.generate_discrete(batch_size*10, reuse=True, 
                                      edge_contact_time=edge_contact_time,n_eval_loop=n_eval_loop)

fake_graphs = []
fake_x_t0 = []
real_walks = []
real_x_t0 = []
for q in range(n_eval_iters):
    fake_outputs, node_logit = tggan.session.run([
        sample_many, tggan.logit], {tggan.temp: 0.5})
    fake_x, fake_t0, fake_edges, fake_t, fake_end = fake_outputs
    smpls = None
    stop = [False] * n_smpls
    for i in range(n_eval_loop):
        x, t0, e, tau, le = fake_x[i], fake_t0[i], fake_edges[i], fake_t[i], fake_end[i]

        if q == 0 and i >= n_eval_loop-3:
            log('eval_iters: {} eval_loop: {}'.format(q, i))
            log('eval node logit min: {} max: {}'.format(node_logit.min(), node_logit.max()))
            log('generated [x, t0, e, tau, end]\n[{}, {}, {}, {}, {}]'.format(
                x[0], t0[0, 0], e[0, :], tau[0, :, 0], le[0]
            ))
            log('generated [x, t0, e, tau, end]\n[{}, {}, {}, {}, {}]'.format(
                x[1], t0[1, 0], e[1, :], tau[1, :, 0], le[1]
            ))

        e = e.reshape(-1, tggan.rw_len, 2)
        tau = tau.reshape(-1, tggan.rw_len, 1)
        if i == 0:
            smpls = np.concatenate([e, tau],axis=-1)
        else:
            new_pred = np.concatenate([e[:, -1:], tau[:, -1:]], axis=-1)
            smpls = np.concatenate([smpls, new_pred], axis=1)

        # judge if reach max length
        for b in range(n_smpls):
            b_le = le[b]

            if i == 0 and b_le == 1:  # end
                stop[b] = True
            if i > 0 and stop[b]:  # end
                smpls[b, -1, :] = -1
            if i > 0 and not stop[b] and b_le == 1:
                stop[b] = True

    fake_x = np.array(fake_x).reshape(-1, 1)
    fake_t0 = np.array(fake_t0).reshape(-1, 1)
    fake_len = np.array(fake_end).reshape(-1, 1) # change to end
    fake_start = np.c_[fake_x, fake_t0, fake_len]
    fake_x_t0.append(fake_start)
    fake_graphs.append(smpls)

2020-06-04 05:09:07,802 - INFO - eval_iters: 0 eval_loop: 22
2020-06-04 05:09:07,803 - INFO - eval node logit min: -1.1131410598754883 max: 1.4473310708999634
2020-06-04 05:09:07,804 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [77 63 58 18 90 69 85 20  4 42 96 99 58 59 12 71], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:09:07,805 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [35 64 95 47  6 94 95 72 82 29 69 90 60 31 77 12], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:09:07,806 - INFO - eval_iters: 0 eval_loop: 23
2020-06-04 05:09:07,807 - INFO - eval node logit min: -1.1131410598754883 max: 1.4473310708999634
2020-06-04 05:09:07,807 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [58 18 90 69 85 20  4 42 96 99 58 59 12 71 12 55], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:09:07,808 - INFO - generated [x, t0, e, tau, end]
[0, 0.0, [95 47  6 94 95 72 82 29 69 90 60 31 77 12 11 22], [0. 0. 0. 0. 0. 0. 0. 0.], 1]
2020-06-04 05:09:07,809 - INFO - eval_iters: 0 eval_loop: 24
2020-

In [14]:
print(np.array(fake_graphs)[0])

[[[34.       77.        0.026708]
  [72.       63.        0.001126]
  [ 4.        9.        0.000019]
  ...
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]]

 [[12.       62.        0.054622]
  [52.       57.        0.005634]
  [32.       41.        0.00013 ]
  ...
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]]

 [[ 3.       42.        0.040148]
  [35.       27.        0.004854]
  [92.       20.        0.000137]
  ...
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]]

 ...

 [[ 5.       45.        0.072849]
  [95.       44.        0.003608]
  [46.       98.        0.000082]
  ...
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]
  [-1.       -1.       -1.      ]]

 [[66.       36.        0.083236]
  [69.       45.        0.009884]
  [98.       17.        0.000261]
  ...
  [-1.       -1.       -1.      ]
  [-

In [7]:
sample_many = tggan.generate_discrete(batch_size*10, reuse=True, 
                                      edge_contact_time=edge_contact_time,n_eval_loop=n_eval_loop)
samples = []
for i in range(1, 2):
    if i % 500 == 0:
        print(i+1)
    samples.append(sample_many.eval({tggan.tau: 0.5}))

AttributeError: 'tuple' object has no attribute 'eval'